In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import timedelta, datetime
import time

In [61]:
df = pd.read_csv('../data/marathon_results_2016.csv', index_col='Bib', 
                 usecols=['Bib','40K','Half','Pace','Age','M/F','Country','State','City'])

In [69]:
df.head()

,Age,M/F,City,State,Country,Half,40K,Pace,TotalSeconds,HalfSeconds
Bib,,,,,,,,,,
5,21,M,Addis Ababa,NaN,ETH,0 days 01:06:45,0 days 02:05:59,0:05:04,7559.0,4005.0
1,26,M,Ambo,NaN,ETH,0 days 01:06:46,0 days 02:05:59,0:05:06,7559.0,4006.0
6,31,M,Addis Ababa,NaN,ETH,0 days 01:06:44,0 days 02:06:47,0:05:07,7607.0,4004.0
11,33,M,Kitale,NaN,KEN,0 days 01:06:46,0 days 02:06:47,0:05:07,7607.0,4006.0
14,23,M,Eldoret,NaN,KEN,0 days 01:06:46,0 days 02:08:11,0:05:11,7691.0,4006.0


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 26630 entries, 5 to 28487
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Age      26630 non-null  int64 
 1   M/F      26630 non-null  object
 2   City     26629 non-null  object
 3   State    23783 non-null  object
 4   Country  26630 non-null  object
 5   Half     26630 non-null  object
 6   40K      26630 non-null  object
 7   Pace     26630 non-null  object
dtypes: int64(1), object(7)
memory usage: 12.7 MB


In [63]:
df['40K'] = df['40K'].replace('-', 0)
df['Half'] = df['Half'].replace('-', 0)

df['40K'] = df['40K'].apply(pd.to_timedelta)
df['Half'] = df['Half'].apply(pd.to_timedelta)
df['TotalSeconds'] = df['40K'].apply(lambda x: timedelta.total_seconds(x))
df['HalfSeconds'] = df['Half'].apply(lambda x: timedelta.total_seconds(x))

In [64]:
#df.sort_index(inplace=True)
df.head()

,Age,M/F,City,State,Country,Half,40K,Pace,TotalSeconds,HalfSeconds
Bib,,,,,,,,,,
5,21,M,Addis Ababa,NaN,ETH,0 days 01:06:45,0 days 02:05:59,0:05:04,7559.0,4005.0
1,26,M,Ambo,NaN,ETH,0 days 01:06:46,0 days 02:05:59,0:05:06,7559.0,4006.0
6,31,M,Addis Ababa,NaN,ETH,0 days 01:06:44,0 days 02:06:47,0:05:07,7607.0,4004.0
11,33,M,Kitale,NaN,KEN,0 days 01:06:46,0 days 02:06:47,0:05:07,7607.0,4006.0
14,23,M,Eldoret,NaN,KEN,0 days 01:06:46,0 days 02:08:11,0:05:11,7691.0,4006.0


In [60]:
df.pivot_table(index='Age', columns='M/F', values='TotalSeconds')

M/F,F,M
Age,,
18,16050.666667,14554.615385
19,15351.958333,12156.529412
20,13835.931818,12111.100000
21,14681.027027,12408.360465
22,14366.421053,11872.666667
...,...,...
78,NaN,16833.714286
79,NaN,14803.500000
80,16211.000000,15755.000000


In [65]:
pivTab = df.pivot_table(index='Age', columns='M/F', values=['TotalSeconds', 'HalfSeconds'], aggfunc='mean')
pivTab.head()

HalfSeconds               TotalSeconds              
M/F            F            M             F             M
Age                                                      
18   7999.888889  7078.538462  16050.666667  14554.615385
19   7473.666667  5817.764706  15351.958333  12156.529412
20   6871.045455  5716.800000  13835.931818  12111.100000
21   7148.554054  5929.627907  14681.027027  12408.360465
22   7091.097744  5751.000000  14366.421053  11872.666667

In [66]:
pivTab = pivTab.swaplevel(0,-1,axis='columns')
pivTab.head()

M/F,F,M,F,M
,HalfSeconds,HalfSeconds,TotalSeconds,TotalSeconds
Age,,,,
18,7999.888889,7078.538462,16050.666667,14554.615385
19,7473.666667,5817.764706,15351.958333,12156.529412
20,6871.045455,5716.800000,13835.931818,12111.100000
21,7148.554054,5929.627907,14681.027027,12408.360465
22,7091.097744,5751.000000,14366.421053,11872.666667


In [41]:
pivTab = pivTab.sort_index(axis='columns')
pivTab.head()

M/F            F                          M              
     HalfSeconds  TotalSeconds  HalfSeconds  TotalSeconds
Age                                                      
18   7999.888889  16050.666667  7078.538462  14554.615385
19   7473.666667  15351.958333  5817.764706  12156.529412
20   6871.045455  13835.931818  5716.800000  12111.100000
21   7148.554054  14681.027027  5929.627907  12408.360465
22   7091.097744  14366.421053  5751.000000  11872.666667

In [46]:
pivTab2 = df.pivot_table(values='City', index='Age', columns='M/F', aggfunc='count')
pivTab2.head()

M/F,F,M
Age,,
18,9.0,13.0
19,24.0,17.0
20,44.0,40.0
21,74.0,86.0
22,133.0,93.0


In [48]:
df.pivot_table(values='City', index='Age', aggfunc='count')

,City
Age,
18,22
19,41
20,84
21,160
22,226
...,...
78,7
79,2
80,4


In [74]:
df.pivot_table(values='40K', index='Country', aggfunc='count').sort_values('40K', ascending=False)

,40K
Country,
USA,21649
CAN,2134
GBR,366
MEX,252
JPN,185
...,...
JOR,1
AND,1
FLK,1


In [75]:
df.pivot_table(index=['Country', 'State', 'City'], values='TotalSeconds', aggfunc='mean').sort_values('TotalSeconds', ascending=False)

TotalSeconds
Country State City                      
USA     WI    Reedsburg          28461.0
        TX    Justin             23816.0
        MS    Olive Branch       21878.0
        FL    Estero             21468.0
        TX    Fresno             20919.0
...                                  ...
        NE    Kearney             8354.0
        NY    New Paltz           8182.0
        PA    Sellersville        6399.0
        CA    Rossmoor               0.0
        MN    Northfield             0.0

[5439 rows x 1 columns]